<a href="https://colab.research.google.com/github/AylinNaebzadeh/MultiLabel-Text-Classification/blob/main/Mental_Health_Topic_Classification_S6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Packages

In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.7 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [3]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00


In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


# Pre-process main csv file

In [5]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM,DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from google.colab import drive
from keras import backend as K
import tensorflow as tf
from tensorflow import keras

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
file = pd.read_csv('/content/drive/MyDrive/primate_data.csv')

In [8]:

df = file

labels = list(df.columns[2:])

new_df = pd.DataFrame(columns=["post_title", "post_text", "label", "value"])

for index, row in df.iterrows():
  for label in labels:
    new_row = {"post_title": row["post_title"], "post_text": row["post_text"], "label": label, "value": row[label]}
    # new_df = new_df.append(new_row, ignore_index=True)
    new_df = pd.concat([new_df, pd.DataFrame(new_row, index=[0])], ignore_index=True)


new_df.to_csv("primate_data_single_label.csv", index=False)

In [9]:
new_df

,post_title,post_text,label,value
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-bad-about-yourself-or-that-you-are-a-f...,1
1,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-down-depressed-or-hopeless,0.0
2,I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-tired-or-having-little-energy,1
3,I don't feel original anymore.,"When I was in high school a few years back, I ...",Little-interest-or-pleasure-in-doing,1
4,I don't feel original anymore.,"When I was in high school a few years back, I ...",Moving-or-speaking-so-slowly-that-other-people...,0
...,...,...,...,...
18022,When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Moving-or-speaking-so-slowly-that-other-people...,0
18023,When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Poor-appetite-or-overeating,0.0
18024,When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Thoughts-that-you-would-be-better-off-dead-or-...,0.0
18025,When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Trouble-concentrating-on-things-such-as-readin...,0.0


In [10]:
df = pd.read_csv('primate_data_single_label.csv', header=None, names=['post_text', 'label', 'value'])

In [11]:
df

,post_text,label,value
post_title,post_text,label,value
I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-bad-about-yourself-or-that-you-are-a-f...,1
I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-down-depressed-or-hopeless,0.0
I don't feel original anymore.,"When I was in high school a few years back, I ...",Feeling-tired-or-having-little-energy,1
I don't feel original anymore.,"When I was in high school a few years back, I ...",Little-interest-or-pleasure-in-doing,1
...,...,...,...
When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Moving-or-speaking-so-slowly-that-other-people...,0
When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Poor-appetite-or-overeating,0.0
When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Thoughts-that-you-would-be-better-off-dead-or-...,0.0
When you're the rock but have no one to lean on.,It's Thanksgiving and I spent the day with my ...,Trouble-concentrating-on-things-such-as-readin...,0.0


In [12]:
labels = df['label'].unique()

In [13]:
labels

array(['label',
       'Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down',
       'Feeling-down-depressed-or-hopeless',
       'Feeling-tired-or-having-little-energy',
       'Little-interest-or-pleasure-in-doing ',
       'Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual',
       'Poor-appetite-or-overeating',
       'Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way',
       'Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television',
       'Trouble-falling-or-staying-asleep-or-sleeping-too-much'],
      dtype=object)

In [14]:
# pre-process the labels
labels = labels[1:]

pre_prcoessed_labels = [label.replace('-', ' ') for label in labels]


In [15]:
pre_prcoessed_labels

['Feeling bad about yourself or that you are a failure or have let yourself or your family down',
 'Feeling down depressed or hopeless',
 'Feeling tired or having little energy',
 'Little interest or pleasure in doing ',
 'Moving or speaking so slowly that other people could have noticed Or the opposite being so fidgety or restless that you have been moving around a lot more than usual',
 'Poor appetite or overeating',
 'Thoughts that you would be better off dead or of hurting yourself in some way',
 'Trouble concentrating on things such as reading the newspaper or watching television',
 'Trouble falling or staying asleep or sleeping too much']

In [16]:
new_df['post_text']

0        When I was in high school a few years back, I ...
1        When I was in high school a few years back, I ...
2        When I was in high school a few years back, I ...
3        When I was in high school a few years back, I ...
4        When I was in high school a few years back, I ...
                               ...                        
18022    It's Thanksgiving and I spent the day with my ...
18023    It's Thanksgiving and I spent the day with my ...
18024    It's Thanksgiving and I spent the day with my ...
18025    It's Thanksgiving and I spent the day with my ...
18026    It's Thanksgiving and I spent the day with my ...
Name: post_text, Length: 18027, dtype: object

In [17]:
# The below code simply appends the two lists together to form a single list of sentences...
# the length of sentences is the number of posts + 9(the number of labels)
sentences = list(new_df['post_text'].unique()) + list(pre_prcoessed_labels)
dataset = Dataset.from_dict({"text": sentences})

In [18]:
sentences[-2]

'Trouble concentrating on things such as reading the newspaper or watching television'

In [19]:
len(sentences)

2011

# Imporint BERT, Training, etc...
https://huggingface.co/bert-base-uncased

In [20]:
bert_model = "bert-base-uncased"

The reason of adding ```padding=True``` and ```truncation=True```, was the below error in training cell:

```RuntimeError: The size of tensor a (1260) must match the size of tensor b (512) at non-singleton dimension 1```

In [21]:
# tokenize dataset (i.e. the "text" column)
tokenizer = BertTokenizer.from_pretrained(bert_model)
tokenized_dataset = dataset.map(lambda x:
                                tokenizer(x["text"], padding=True, truncation=True), batched=True, num_proc=4, remove_columns=["text"])
# load model
model = AutoModelForMaskedLM.from_pretrained(bert_model)
# define some stuff for the training
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Map (num_proc=4):   0%|          | 0/2011 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
training_args = TrainingArguments(
    "test-clm",
    save_total_limit=20,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    data_collator=data_collator,
)
# train
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,1.822924
2,1.976700,1.744400
3,1.976700,1.741764


TrainOutput(global_step=756, training_loss=1.931946769593254, metrics={'train_runtime': 1007.1309, 'train_samples_per_second': 5.99, 'train_steps_per_second': 0.751, 'total_flos': 1587914635622400.0, 'train_loss': 1.931946769593254, 'epoch': 3.0})

In [23]:
fine_tuned_checkpoint = f"model/{bert_model}_ft"

In [ ]:
fine_tuned_checkpoint_google_drive = f"/content/drive/MyDrive/fine_tuned_models/model/{bert_model}_ft"


In [25]:
# save model + tokenizer
trainer.save_model(fine_tuned_checkpoint)
tokenizer.save_pretrained(fine_tuned_checkpoint)

('model/bert-base-uncased_ft/tokenizer_config.json',
 'model/bert-base-uncased_ft/special_tokens_map.json',
 'model/bert-base-uncased_ft/vocab.txt',
 'model/bert-base-uncased_ft/added_tokens.json')

# BERT Text Preparation

In [24]:
def bert_text_preparation(text, tokenizer):
  """
  Preprocesses text input in a way that BERT can interpret.
  """
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1]*len(indexed_tokens)
  # convert inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensor = torch.tensor([segments_ids])
  return tokenized_text, tokens_tensor, segments_tensor

In [26]:
def get_bert_embeddings(tokens_tensor, segments_tensor, model):
  """
  Obtains BERT embeddings for tokens.
  """
  # gradient calculation id disabled
  with torch.no_grad():
    # obtain hidden states
    outputs = model(tokens_tensor, segments_tensor)
    hidden_states = outputs[2]
  # concatenate the tensors for all layers
  # use "stack" to create new dimension in tensor
  token_embeddings = torch.stack(hidden_states, dim=0)
  # remove dimension 1, the "batches"
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # swap dimensions 0 and 1 so we can loop over tokens
  token_embeddings = token_embeddings.permute(1,0,2)
  # intialized list to store embeddings
  token_vecs_sum = []
  # "token_embeddings" is a [Y x 12 x 768] tensor
  # where Y is the number of tokens in the sentence
  # loop over tokens in sentence
  for token in token_embeddings:
  # "token" is a [12 x 768] tensor
  # sum the vectors from the last four layers
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
  return token_vecs_sum

In [27]:
bert_model = BertModel.from_pretrained(fine_tuned_checkpoint, output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained(fine_tuned_checkpoint)

Some weights of BertModel were not initialized from the model checkpoint at model/bert-base-uncased_ft and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
label_embeddings = {}
for label in pre_prcoessed_labels:
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(label, tokenizer)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, bert_model)
  label_embedding = torch.stack(list_token_embeddings, dim=0).sum(dim=0)
  label_embeddings[label] = label_embedding

In [29]:
label

'Trouble falling or staying asleep or sleeping too much'

In [30]:
segments_tensors

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [31]:
len(list_token_embeddings)

11

In [32]:
sum(list_token_embeddings[:][0]).item() / len(tokens_tensor[0])

-6.382847872647372

In [ ]:
for item in list_token_embeddings[0]

In [33]:
len(label_embedding) * 2

1536

# Train MLP

In [34]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [35]:
mlp_model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[1536,]),
    # keras.layers.Dense(1000, activation='relu'),
    # keras.layers.Dense(500, activation='relu'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [36]:
mlp_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer = 'adam',
                  metrics=['accuracy', f1_m])

In [ ]:
file.head()

,post_title,post_text,Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down,Feeling-down-depressed-or-hopeless,Feeling-tired-or-having-little-energy,Little-interest-or-pleasure-in-doing,Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual,Poor-appetite-or-overeating,Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way,Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television,Trouble-falling-or-staying-asleep-or-sleeping-too-much
0,I don't feel original anymore.,"When I was in high school a few years back, I ...",1,0.0,1,1,0,0.0,0.0,0.0,0
1,I don’t have anyone to talk to and I don’t kno...,Nine years ago I was diagnosed with depression...,1,1.0,0,0,1,0.0,0.0,0.0,0
2,Stuck between moving forward and killing myself,Some background information: My GF of almost 3...,1,1.0,1,1,0,0.0,1.0,0.0,0
3,I need help on how to help my girlfriend.,"My girlfriend ,of about 3 months now ,has been...",1,1.0,0,0,0,0.0,1.0,0.0,0
4,I've been feeling this way for some tine,I'm alway feeling like this. It doesn't even m...,1,1.0,0,0,0,0.0,0.0,0.0,0


In [37]:
!pip install tqdm

In [38]:
from tqdm.notebook import tqdm_notebook
X = []
y = []
for index, row in tqdm_notebook(new_df.iterrows()):
  try:
    # print("********* TRY ********")
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(row['post_text'], tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, bert_model)
    question_embedding = torch.stack(list_token_embeddings, dim=0).sum(dim=0)
    # print(row)
    label_embedding = label_embeddings[row['label'].replace('-', ' ')]
    x = torch.cat((question_embedding, label_embedding), 0).numpy()
    X.append(x)
    y.append(int(row['value']))
  except:
    # print("********* EXCEPT ********")
    continue

0it [00:00, ?it/s]

In [45]:
type(y)

list

In [46]:
y

[1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,


In [44]:
type(X)

list

In [47]:
X

[array([  0.900383, 193.26678 , 168.7979  , ..., -28.939186,   0.858171,
        -15.650099], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ..., -10.126764 ,
        -14.588393 ,  -7.3978443], dtype=float32),
 array([  0.900383, 193.26678 , 168.7979  , ...,  -9.411978, -20.002766,
         -9.58005 ], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ...,  -5.073497 ,
        -14.488961 ,  -5.8189487], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ..., -57.431904 ,
         -3.1115437,  -7.0367594], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ...,  -6.0879507,
        -23.155718 , -16.440035 ], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ..., -20.988651 ,
         -4.5537577,   9.806619 ], dtype=float32),
 array([  0.900383 , 193.26678  , 168.7979   , ...,  -7.6693153,
         -6.9336414, -13.453826 ], dtype=float32),
 array([  0.900383, 193.26678 , 168.7979  , ..., -12.090614, -24.634846,
         -9

In [48]:
X_tf = tf.convert_to_tensor(X) # or X = np.array(X)
y_tf = tf.convert_to_tensor(y) # or y = np.array(y)

In [50]:
y_tf

<tf.Tensor: shape=(16258,), dtype=int32, numpy=array([1, 0, 1, ..., 0, 0, 0], dtype=int32)>

In [49]:
X_tf

<tf.Tensor: shape=(16263, 1536), dtype=float32, numpy=
array([[   0.900383 ,  193.26678  ,  168.7979   , ...,  -28.939186 ,
           0.858171 ,  -15.650099 ],
       [   0.900383 ,  193.26678  ,  168.7979   , ...,  -10.126764 ,
         -14.588393 ,   -7.3978443],
       [   0.900383 ,  193.26678  ,  168.7979   , ...,   -9.411978 ,
         -20.002766 ,   -9.58005  ],
       ...,
       [-115.32484  ,  113.73878  ,  167.86516  , ...,  -20.988651 ,
          -4.5537577,    9.806619 ],
       [-115.32484  ,  113.73878  ,  167.86516  , ...,   -7.6693153,
          -6.9336414,  -13.453826 ],
       [-115.32484  ,  113.73878  ,  167.86516  , ...,  -12.090614 ,
         -24.634846 ,   -9.919979 ]], dtype=float32)>

In [52]:
# Assume y is a tensor with shape (16258,)
y_tf = tf.random.uniform((16258,), minval=0, maxval=2, dtype=tf.int32)

# Pad y with 0 values at the end of the first dimension
y_tf = tf.pad(y, [[0, 5]])

# Print the new shape of y
print(y_tf.shape)

(16263,)


In [53]:
history = mlp_model.fit(X_tf, y_tf, epochs=20, validation_split=0.2)

Epoch 1/20
407/407 [==============================] - 5s 11ms/step - loss: 19.5986 - accuracy: 0.6242 - f1_m: 0.0105 - val_loss: 14.9406 - val_accuracy: 0.6351 - val_f1_m: 0.0173
Epoch 2/20
407/407 [==============================] - 2s 5ms/step - loss: 16.4860 - accuracy: 0.6225 - f1_m: 0.0064 - val_loss: 13.2884 - val_accuracy: 0.6287 - val_f1_m: 0.0071
Epoch 3/20
407/407 [==============================] - 2s 5ms/step - loss: 15.6924 - accuracy: 0.6226 - f1_m: 0.0064 - val_loss: 11.5758 - val_accuracy: 0.6216 - val_f1_m: 0.0000e+00
Epoch 4/20
407/407 [==============================] - 2s 5ms/step - loss: 18.4742 - accuracy: 0.6262 - f1_m: 0.0081 - val_loss: 16.9052 - val_accuracy: 0.6287 - val_f1_m: 0.0028
Epoch 5/20
407/407 [==============================] - 2s 5ms/step - loss: 16.7080 - accuracy: 0.6243 - f1_m: 0.0073 - val_loss: 12.0297 - val_accuracy: 0.6225 - val_f1_m: 0.0000e+00
Epoch 6/20
407/407 [==============================] - 3s 8ms/step - loss: 16.5561 - accuracy: 0.6228 